In [1]:
import geopandas as gpd
import os
import pandas as pd

In [2]:
path = r'T:\Trans Projects\Model Development\UrbanSim_LandUse\Output\Simulation_47_Final_RTP'

In [3]:
shp = 'parcel_fabric.shp'

In [166]:
parcel_data = gpd.read_file(os.path.join(path, shp))

In [5]:
parcel_data.columns

Index(['btype', 'nrsqft', 'rsqft', 'du', 'yrbuilt', 'lpid', 'pundev',
       'dev_land', 'developed', 'obtype', 'orsqft', 'onrsqft', 'odu', 'zid',
       'rezoned', 'city', 'annexed', 'ozid', 'geometry'],
      dtype='object')

In [6]:
parcel_data.head()

,btype,nrsqft,rsqft,du,yrbuilt,lpid,pundev,dev_land,developed,obtype,orsqft,onrsqft,odu,zid,rezoned,city,annexed,ozid,geometry
0,(),0.0,0.0,0.0,0.0,24284,0.00,1.123953e+04,0,(),0.0,0.0,0.0,119,0,None,0,119,"MULTIPOLYGON (((4254437.869 843733.606, 425444..."
1,"(5100,)",1662.0,0.0,0.0,1969.0,24356,0.15,1.973955e+06,0,"(5100,)",0.0,1662.0,0.0,119,0,None,0,119,"POLYGON ((4254479.021 845048.443, 4254437.869 ..."
2,"(1110,)",0.0,3468.0,1.0,2006.0,24391,0.00,1.099604e+05,0,"(1110,)",3468.0,0.0,1.0,62,0,None,0,62,"POLYGON ((4250857.967 845652.695, 4250847.053 ..."
3,"(1110,)",0.0,1296.0,1.0,1984.0,24408,0.00,6.144933e+04,0,"(1110,)",1296.0,0.0,1.0,62,0,None,0,62,"POLYGON ((4250865.647 845846.211, 4250859.554 ..."
4,"(1110,)",0.0,2528.0,1.0,2015.0,24419,0.00,6.097905e+04,0,"(1110,)",2528.0,0.0,1.0,62,0,None,0,62,"POLYGON ((4250871.702 845998.498, 4250865.647 ..."


In [7]:
parcel_data.shape

(78216, 19)

In [8]:
parcel_data[parcel_data['yrbuilt'] > 2020]['odu'].describe()

count    2703.000000
mean        0.912320
std        10.638641
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max       536.000000
Name: odu, dtype: float64

In [9]:
parcel_data[parcel_data['yrbuilt'] < 2020]['du'].describe()

count    75513.000000
mean         1.495623
std          7.124226
min          0.000000
25%          1.000000
50%          1.000000
75%          1.000000
max        400.000000
Name: du, dtype: float64

In [10]:
parcel_data[parcel_data['yrbuilt'] < 2020]['nrsqft'].describe()

count    7.551300e+04
mean     9.496663e+02
std      1.209315e+04
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.117553e+06
Name: nrsqft, dtype: float64

In [11]:
parcel_data[parcel_data['yrbuilt'] > 2020]['onrsqft'].describe()

count      2703.000000
mean       2190.273400
std       10715.194805
min           0.000000
25%           0.000000
50%           0.000000
75%        2295.000000
max      475132.000000
Name: onrsqft, dtype: float64

In [12]:
parcel_data['btype'].dtypes

dtype('O')

In [13]:
parcel_data[['btype']].head()

,btype
0,()
1,"(5100,)"
2,"(1110,)"
3,"(1110,)"
4,"(1110,)"


In [14]:
parcel_data.btype.apply(len).max()

36

In [15]:
#parcel_data['btype_len'] = parcel_data.btype.apply(len)
#parcel_data.loc[parcel_data.btype_len == parcel_data.btype_len.max(), 'btype']

24842    (1110, 4100, 5100, 6120, 6500, 7100)
Name: btype, dtype: object

In [16]:
#parcel_data['btype1'] = parcel_data['btype'].apply(lambda x: str(x)[1:-2].split(', ')[0])

In [17]:
#https://github.com/urbansim/lcog/blob/master/lcog/data/building_types.csv
df = pd.read_csv('building_types.csv')

In [18]:
df.head()

,building_type_id,name,description,is_residential,is_non_residential,is_other
0,1110,Single-family Detached,Single Family Detached,True,False,False
1,1121,Townhome or Rowhouse,"Single-Family Attached, Townhome or Rowhouse",True,False,False
2,1122,Duplex,"Single-Family Attached, Duplex",True,False,False
3,1130,Mobile Homes,"Single Family, Mobile Home",True,False,False
4,1210,Multifamily - Small Building,"Multifamily, Triplex or Small (3 to 19 unit) A...",True,False,False


In [19]:
btypes = df[df['is_non_residential']]['building_type_id'].unique()

In [122]:
ndf = df[df['is_residential']]
res_btypes = ndf[~ndf['is_non_residential']]['building_type_id'].unique()

In [20]:
#https://github.com/urbansim/lcog/blob/master/lcog/data/bsqft_per_job.csv
bsqft_per_job = pd.read_csv('bsqft_per_job.csv')

In [21]:
bsqft_per_job.head()

,building_type_id,area_per_job,placeholder_area_per_job
0,5200,624.844490,0
1,5100,720.152698,0
2,9000,112.675926,0
3,4220,170.982468,0
4,4320,568.986339,0


In [161]:
def area_per_job(x):
    if x[1:-2] == '':
        area_per_job = None
        isNonRes = False
    else:
        if len(x[1:-2].split(', ')) == 1:
            a = x[1:-2].split(', ')
            if int(a[0]) in btypes:            
                area_per_job = bsqft_per_job.loc[bsqft_per_job.building_type_id == int(a[0]), 'area_per_job'].values[0]
                isNonRes = True
            elif int(a[0]) in res_btypes:
                if int(a[0]) in bsqft_per_job.building_type_id:
                    area_per_job = bsqft_per_job.loc[bsqft_per_job.building_type_id == int(a[0]), 'area_per_job'].values[0]
                else:
                    area_per_job = None
                isNonRes = False
            else:
                area_per_job = None
                isNonRes = False
        else:
            a = x[1:-1].split(', ')
            b = [int(btype) for btype in a]
            btypeList = [btype for btype in b if btype in btypes]
            res_btypeList = [btype for btype in b if btype in res_btypes]
            if btypeList == []:
                area_per_job = bsqft_per_job.loc[bsqft_per_job.building_type_id.isin(res_btypeList), 'area_per_job'].mean()
                isNonRes = False
            else:
                area_per_job = bsqft_per_job.loc[bsqft_per_job.building_type_id.isin(btypeList), 'area_per_job'].mean()
                isNonRes = True
    return(area_per_job, isNonRes)   

In [167]:
parcel_data['AreaPerJob'] = parcel_data['btype'].apply(lambda x: area_per_job(x)[0])

In [168]:
parcel_data['isNonRes'] = parcel_data['btype'].apply(lambda x: area_per_job(x)[1])

In [183]:
def compute_jobs(x, sqft, area):
    return sqft / area if x else None

In [192]:
parcel_data.jobs = parcel_data.apply(lambda row: compute_jobs(row.isNonRes, row.nrsqft, row.AreaPerJob), axis=1)
parcel_data.ojobs = parcel_data.apply(lambda row: compute_jobs(row.isNonRes, row.onrsqft, row.AreaPerJob), axis=1)

In [190]:
parcel_data['jobs'].max()

6072.616755069882

In [191]:
parcel_data['jobs'].min()

0.0

In [181]:
# https://github.com/urbansim/lcog/blob/master/lcog/data/target_vacancies.csv
parcel_data['hh'] = parcel_data['du'].apply(lambda x: x*0.965)
parcel_data['ohh'] = parcel_data['odu'].apply(lambda x: x*0.965)

In [193]:
parcel_data.to_file(os.path.join(path, 'output', 'parcel_data.shp'))

In [194]:
parcel_data.columns

Index(['btype', 'nrsqft', 'rsqft', 'du', 'yrbuilt', 'lpid', 'pundev',
       'dev_land', 'developed', 'obtype', 'orsqft', 'onrsqft', 'odu', 'zid',
       'rezoned', 'city', 'annexed', 'ozid', 'geometry', 'AreaPerJob',
       'isNonRes', 'jobs', 'ojobs', 'hh', 'ohh'],
      dtype='object')

In [195]:
baseyearJobs = parcel_data[parcel_data['ojobs'] > 0]

In [196]:
baseyearJobs.to_file(os.path.join(path, 'output', 'baseyearJobs.shp'))

In [197]:
forecastJobs = parcel_data[parcel_data['jobs'] > 0]

In [198]:
forecastJobs.to_file(os.path.join(path, 'output', 'forecastJobs.shp'))

In [200]:
parcel_data.shape

(78216, 25)

In [201]:
baseyearJobs.shape

(3450, 25)

In [202]:
forecastJobs.shape

(3854, 25)

In [203]:
parcel_data[parcel_data['hh'] > 0].shape

(72627, 25)

In [204]:
parcel_data[parcel_data['ohh'] > 0].shape

(72078, 25)

In [205]:
baseyearJobs.ojobs.describe()

count    3450.000000
mean       44.076939
std       122.082840
min         0.207834
25%         8.249104
50%        18.118844
75%        40.326603
max      2414.868846
Name: ojobs, dtype: float64

In [206]:
baseyearJobs.jobs.describe()

count    3450.000000
mean       57.176254
std       171.308358
min         0.207834
25%         9.285145
50%        22.424411
75%        52.770532
max      6072.616755
Name: jobs, dtype: float64